需求：通过名称访问序列元素，以减少结构对位置的依赖性

In [12]:
# 相比普通的元组，collections.namedtuple()(命名元组)开销小，当便利。实际上namedtuple()是一个工厂方法，它返回
# python中标准元组类型的子类。只要提供一个类型名称，及相应的字段即可。

from collections import namedtuple

# 实例化
Subscriber = namedtuple('Subscriber', ['addr', 'joined'])
sub = Subscriber('abc@expample.com', '2018-1-22')
print(sub)
print(sub.addr)
print(sub.joined)

# 实例化其他写法
Subscriber = namedtuple('Subscriber', 'addr joined')
print(Subscriber)
Subscriber = namedtuple('Subscriber', 'addr,joined')
print(Subscriber)


Subscriber(addr='abc@expample.com', joined='2018-1-22')
abc@expample.com
2018-1-22
<class '__main__.Subscriber'>
<class '__main__.Subscriber'>


In [13]:
# namedtuple生成的元组，支持普通元组的操作。

print(len(sub))

addr, joined = sub
print(addr)
print(joined)


2
abc@expample.com
2018-1-22


In [14]:
# 命名元组的主要作用在于将代码同它所控制的元素位置解耦。所以对于从数据库，或从csv中读取数据，可以通过命名元组
# 来组织代码

# 实例1：普通代码
def compute_cost(records):
    total = 0.0
    for rec in records:
        total += rec[1] * rec[2]
    return total

# 实例1：命名元组版(如果records中元素是某个类的实例，就不需要转换成命名元组，直接引用属性就可以了)
from collections import namedtuple

Stock = namedtuple('Stock', ['name', 'shares', 'price'])
def compute_cost(records):
    total = 0.0
    for rec in records:
        s = Stock(*rec)
        total += s.shares * s.price
    return total


In [15]:
# namedtuple是一种可能用法是作为字典的替代，后者需要更多空间存储。因此如果涉及构建大型的字典的数据结构，
# 使用nametuple会更效率。但是注意的是，namedtuple是不可变的。

s = Stock('ACME', shares=100, price=123.45)
s.shares = 75 # wrong

AttributeError: can't set attribute

In [16]:
# 如果需要修改任何属性，可以通过namedtuple实例的_replace()方法来实现。
# 该方法会创建一个全新的命名元组，并对相应的值做替换。

s = s._replace(shares=75)
print(s)


Stock(name='ACME', shares=75, price=123.45)


In [19]:
# _replace()方法还有一个微妙的用法，那就是他可以作为一种简便的方法填充具有可选或缺失字段的命名元组。
# 要做到这点，首先得创建一个包含默认值的"原型"元组，然后使用_replace()方法创建新的实例，替换旧值。

from collections import namedtuple

Stock = namedtuple('Stock', 'name,shares,price,date,time')

# 创建原型
stock_prototype = Stock('', 0, 0.0, None, None)

# 将字典转换成Stock函数
def dict_to_stock(s):
    return stock_prototype._replace(**s)

# 用法
a = {'name': 'ACME', 'shares': 100, 'price': 123.45}
dict_to_stock(a)


Stock(name='ACME', shares=100, price=123.45, date=None, time=None)

In [21]:
"""
    补充：namedtuple其他用法
    
    语法：
    collections.namedtuple(typename, field_names, *, verbose=False, rename=False, module=None)
        参数：
        - typename：元组名
        - field_names:字段名，可用['X','Y'] 或'X Y' 或'X,Y'表示，不能以下划线，数字开头，及使用python保留字
        - verbose: 如果是True,会打印类的信息
        - rename：如果是True,当字段名称无效时，会用位置名替代。如：
            ['abc', 'def', 'ghi', 'abc'] is converted to ['abc', '_1', 'ghi', '_3']
"""

# 元组转OrderedDict()
b = s._asdict()
print(b)

# 通过现有的序列或可迭代对象，生成实例
t = ['AA', '100', 111.11]
c = s._make(t)
print(c)

# 读取csv文件，将其转换成namedtuple元组
EmployeeRecord = namedtuple('EmployeeRecord', 'name, age, title, department, paygrade')

import csv

for emp in map(EmployeeRecord._make, csv.reader(open("employees.csv", "rb"))):
    print(emp.name, emp.title)




OrderedDict([('name', 'ACME'), ('shares', 75), ('price', 123.45)])
Stock(name='AA', shares='100', price=111.11)
